# ESA Fire Climate Change Initiative

## Einleitung

![Sächsische Schweiz](doc/saechsische_schweiz.jpg)

*Waldbrand Sächsische Schweiz [1]*

Seit Juli sind hunderte Einsatzkräfte damit beschäftigt Waldbrände in Sachsen und Brandenburg zu bekämpfen. Auch wenn Waldbrände meist durch fahrlässiges Handeln oder Brandstiftung verursacht werden, ist die steigende Waldbrandgefahr eine Folge des Klimawandels [2]. Um die Entwicklung genauer nachvollziehen zu können, soll in dieser Projektarbeit Daten zum Waldbrandgeschehen betracht werden. 

Das Open Dataset zu den Waldbränden stammt von Website der „ESA Climate Change Initiative“ [3]. Auf der Seite stehen vorverarbeitet Daten eines "Moderate-resolution Imaging Spectroradiometers" (MODIS) zur Verfügung. 
Die Daten liegen im .nc-Format (Network Common Data Format) vor und enthalten pro Rasterzelle (Auflösung 0,25° x 0,25°) der Aufnahme Informationen z.B. zur insgesamt verbrannten Fläche.  


![Dateien](doc/dateien.jpg)

*Download-Bereich ESA Climate Change Initiative*

Ziel der Projektarbeit ist es, ein Teil dieser Daten durch die Nutzung von ``numpy`` und ``pandas`` umzuformatieren, in eine Datenbank zu überführen und auf dieser Basis gezielter Abfragen explorative Analysen durchzuführen (siehe [Vorgehensweise](#Vorgehensweise)). 

## Erste Schritte

In [1]:
import netCDF4 as nc
import pandas as pd
import numpy as np
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')


In [2]:
fn = 'data/20160107-ESACCI-L4_FIRE-BA-MODIS-fv5.0.nc'
ds = nc.Dataset(fn)

Zum Einlesen des Dateiformats wird die „netCDF4“ Bibliothek benötigt. 

### Übersicht Variablen

Folgende Variablen sind in dem Datensatz erhalten: 

In [3]:
list_variables = list()
for var in ds.variables:
    list_variables.append(var)
list_variables

['lat',
 'lat_bnds',
 'lon',
 'lon_bnds',
 'time',
 'time_bnds',
 'vegetation_class',
 'vegetation_class_name',
 'burned_area',
 'standard_error',
 'fraction_of_burnable_area',
 'fraction_of_observed_area',
 'number_of_patches',
 'burned_area_in_vegetation_class']

In [4]:
#explore variables
for var in list_variables: 
    array = ds[var][:]
    print(f" Variable {var} mit Shape {array.shape}")

 Variable lat mit Shape (720,)
 Variable lat_bnds mit Shape (720, 2)
 Variable lon mit Shape (1440,)
 Variable lon_bnds mit Shape (1440, 2)
 Variable time mit Shape (1,)
 Variable time_bnds mit Shape (1, 2)
 Variable vegetation_class mit Shape (18,)
 Variable vegetation_class_name mit Shape (18, 150)
 Variable burned_area mit Shape (1, 720, 1440)
 Variable standard_error mit Shape (1, 720, 1440)
 Variable fraction_of_burnable_area mit Shape (1, 720, 1440)
 Variable fraction_of_observed_area mit Shape (1, 720, 1440)
 Variable number_of_patches mit Shape (1, 720, 1440)
 Variable burned_area_in_vegetation_class mit Shape (1, 18, 720, 1440)


Die verschiedenen Werte der Variablen sind in unterschiedlich dimensionalen Arrays gespeichert. Dies muss für die weitere Verarbeitung berücksichtigt werden. 

### Erstellung Dataframe

Um auf Basis der Geo-Koordinaten einen zwei dimensionalen Dataframe erstellen zu können, sind folgende Schritte notwendig: 

(1) Auslesen der Werte-Arrays aus dem Data Set

In [5]:
lat = ds['lat'][:]
lon = ds['lon'][:]
burned_area = ds['burned_area'][:]
burned_area = burned_area[0]
lon

masked_array(data=[-179.875, -179.625, -179.375, ...,  179.375,  179.625,
                    179.875],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

(2)  Umwandlung der Koordinaten-Vektoren in eine Koordinaten-Matrix:

In [6]:
lons, lats = np.meshgrid(lon, lat)
lons

masked_array(
  data=[[-179.875, -179.625, -179.375, ...,  179.375,  179.625,  179.875],
        [-179.875, -179.625, -179.375, ...,  179.375,  179.625,  179.875],
        [-179.875, -179.625, -179.375, ...,  179.375,  179.625,  179.875],
        ...,
        [-179.875, -179.625, -179.375, ...,  179.375,  179.625,  179.875],
        [-179.875, -179.625, -179.375, ...,  179.375,  179.625,  179.875],
        [-179.875, -179.625, -179.375, ...,  179.375,  179.625,  179.875]],
  mask=False,
  fill_value=1e+20,
  dtype=float32)

(3) Runterbrechen der 2-dimensionalen Matrizen auf ein 1-dimensionalen Array. 

In [7]:
lons_flatt = lons.flatten()
lats_flatt = lats.flatten()
burned_area_flatt= burned_area.flatten()
lons_flatt

masked_array(data=[-179.875, -179.625, -179.375, ...,  179.375,  179.625,
                    179.875],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [8]:
whole_world = {
    'lon': lons_flatt, 
    'lat': lats_flatt, 
    'burned_area' : burned_area_flatt,
}
df_whole_world = pd.DataFrame(whole_world)

In [9]:
df_burned  = df_whole_world[df_whole_world['burned_area'] > 0]
df_burned

,lon,lat,burned_area
268534,-6.375,43.375,107329.0
268537,-5.625,43.375,107329.0
268538,-5.375,43.375,160994.0
268539,-5.125,43.375,643976.0
268544,-3.875,43.375,53664.0
...,...,...,...
778051,-67.125,-45.125,3273545.0
779491,-67.125,-45.375,107329.0
792431,-72.125,-47.625,536646.0
796761,-69.625,-48.375,375652.0


Die Arrays können so in einen Dataframe überführt werden, wobei der Fokus auf den Datenpunkten (Rasterzellen) liegt, in welchen sich tatsächlich „burned area“ befindet. 

### Erste Visualisierung 

In [10]:
fig = go.Figure(data=go.Scattergeo(
        lon = df_burned['lon'],
        lat = df_burned['lat'],
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'Blues',
            cmin = df_burned['burned_area'].min(),
            color = df_burned['burned_area'],
            cmax = df_burned['burned_area'].max(),
            colorbar_title="Burned Area Total"
        )))

fig.update_layout(
        title = 'Burned Areas',
    )
fig.show()


Durch eine erste Visualisierung wird ein Überblick über die Daten erzielt und die Datenverarbeitung kann plausiblisiert werden. 

## Vorgehensweise

Folgende Schritte sollen innerhalb der Projektarbeit umgesetzt weren: 

1.	Dataframe (siehe [Erstellung Dataframe](#erstellung-dataframe)) um weitere Variablen aus dem ursprünglichen Daten erweitern, z.B. fraction_of_burnable_area
2.	Dataframe mit Attributen zu Kontinent und Land anreichern (z.B. mit Bibliothek ``shaply``)
3.	Überprüfung durchführen, ob weitere berechnete Attribute notwendig sind
4. (ggf.Attribute berechnen)
5.	Dataframe in SQL-Datenbank exportieren
6.	Zeitliche Dimension, durch die Durchführung der Schritt 1-6 für weitere Dateien, zur Datenbank ergänzen
7.	Erstellung verschiedener Abfragen auf die Datenbank
8.	Explorative Datenanalyse der Daten 


## Quellen

[1]: https://www.spiegel.de/politik/deutschland/sachsen-michael-kretschmer-bricht-urlaub-wegen-waldbraenden-ab-a-9cde7363-0b93-4a5a-a711-ff7c5391858e, Datum des Aufrufs: 07.08.2022

[2]: https://de.statista.com/infografik/18581/anzahl-und-flaeche-aller-waldbraende-in-deutschland/, Datum des Aufrufs: 07.08.2022

[3]: https://climate.esa.int/en/odp/#/dashboard, Datum des Aufrufs: 07.08.2022
